### Wrangle Exercises

This exercises uses the ```case.csv```, ```dept.csv```, and ```source.csv``` files from the San Antonio 311 call dataset.

#### Data Acquisition

**1. Read the case, department, and source data into their own spark dataframes.**

In [40]:
import pandas as pd
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date

#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [15]:
# read each csv into its own spark dataframe
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

**2. Let's see how writing to the local disk works in spark:**

- Write the code necessary to store the source data in both csv and json format, store these as ```sources_csv``` and ```sources_json```
- Inspect your folder structure. What do you notice?

<font color = 'blue'> A separate directory was generated to store the json and csv files.

In [16]:
# write to json
source.write.json("source_json", mode="overwrite")

# write to csv
source.write.csv("source_csv", mode="overwrite")

**3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.**

In [17]:
source.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [18]:
#look at schema to inspect data types
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [23]:
#cast source_id as an int
source = source.withColumn('source_id', col('source_id').cast('int'))

source.printSchema()

root
 |-- source_id: integer (nullable = true)
 |-- source_username: string (nullable = true)



#### Data Wrangle

In [27]:
# view list of cases
case.show(1,vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



**1. How old is the latest (in terms of days past SLA) currently open issue?**

In [36]:
case.where(case.case_status == 'Open').sort(asc('case_opened_date')).show(1, vertical = True)


-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 1/1/17 13:48         
 case_closed_date     | null                 
 SLA_due_date         | 1/17/17 8:30         
 case_late            | YES                  
 num_days_late        | 348.6458333          
 case_closed          | NO                   
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
only showing top 1 row



In [32]:
case.groupby('SLA_days').count().sort(desc('SLA_days')).show(1)

+--------+-----+
|SLA_days|count|
+--------+-----+
|  1420.0|    1|
+--------+-----+
only showing top 1 row



In [37]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [44]:
from pyspark.sql import SparkSession

case.select(col('case_opened_date'),to_date(col('case_opened_date'),"m/d/yy").alias('case_opened')).show()

Py4JJavaError: An error occurred while calling o199.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 1 times, most recent failure: Lost task 0.0 in stage 47.0 (TID 523) (192.168.1.18 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '1/1/18 0:42' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$an

**How long has the oldest (in terms of days since opened) currently opened issue been open?**

In [39]:
case.withColumn("age_in_days", datediff(current_timestamp(), col("case_opened_date"))
).show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 age_in_days          | null                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO        

**2.  How many Stray Animal cases are there?**

In [30]:
# show a count of cases where the service request type is stray animal
case.where(case.service_request_type == 'Stray Animal').count()

26760

**3. How many service requests that are assigned to the Field Operations department (```dept_division```) are not classified as "Officer Standby" request type (```service_request_type```)?**

In [48]:
case.groupby('service_request_type').count().sort(desc('service_request_type')).show(1, truncate = False)

+--------------------------+-----+
|service_request_type      |count|
+--------------------------+-----+
|Zoning: Visual Obstruction|1859 |
+--------------------------+-----+
only showing top 1 row



**4. Convert the ```council_district``` column to a string column.**

**5. Extract the year from the ```case_closed_date``` column.**

**6. Convert ```num_days_late``` from days to hours in new columns ```num_hours_late```.**

**7. Join the case data with the source and department data.**

**8. Are there any cases that do not have a request source?**

**9. What are the top 10 service request types in terms of number of requests?**

**10. What are the top 10 service request types in terms of average days late?**

**11. Does number of days late depend on department?**

**12. How do number of days late depend on department and request type?**

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.